https://www.tensorflow.org/lite/examples/pose_estimation/overview?hl=ja

https://blog.tensorflow.org/2021/08/pose-estimation-and-classification-on-edge-devices-with-MoveNet-and-TensorFlow-Lite.html?hl=ja&_gl=1*xgz6jk*_ga*MTYwMDI2MzYzMC4xNzA0MTcwODE4*_ga_W0YLR4190T*MTcwNjUxOTU5NC41LjEuMTcwNjUyMDE0OC4wLjAuMA..

TFlite MOvenet RasPi sample code

https://github.com/tensorflow/examples/tree/master/lite/examples/pose_estimation/raspberry_pi#run-the-pose-classification-sample

Install dependencies. 
Run this script to install the Python dependencies, and download the TFLite models. sh setup.sh

for the error following:
ERROR: Could not find a version that satisfies the requirement tflite-runtime>=2.7.0 (from versions: none)
https://temcee.hatenablog.com/entry/tensorflow_install_error

In [2]:
! sh setup.sh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 12.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [259 lines of output]
      Ignoring numpy: markers 'python_version == "3.6" and platform_machine != "aarch64" and platform_machine != "arm64"' don't match your environment
      Ignoring numpy: markers 'python_version >= "3.6" and sys_platform == "linux" and platform_machine == "aarch64"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_machine != "aarch64" and platform_machine != "arm64"' don't match your environment
      Ignoring numpy: mark

-e Downloaded files are in ./


Mac OpenCV install https://qiita.com/makki_maki04/items/f62c5e4e68709d9b3b89
Mac Git install https://qiita.com/suke_masa/items/4bed855628f7414293f8
Homebrew install https://brew.sh/
Mac Tensorflow install https://www.tensorflow.org/install/pip?hl=ja#macos

In [ ]:
python3 pose_estimation.py

In [ ]:
! python3 pose_estimation.py

上記部分はmodule not found errorがでていたがカーネル再起動で通った

In [1]:
"""Main script to run pose classification and pose estimation."""
import argparse
import logging
import sys
import time

import cv2

import utils
from ml import Classifier
from ml import Movenet
from ml import MoveNetMultiPose
from ml import Posenet

In [2]:
def myRun(camera_id: int, width: int, height: int) -> None:
    model = 'movenet_multipose'
    pose_detector = MoveNetMultiPose(model,'bounding_box',512)

    # Variables to calculate FPS
    counter, fps = 0, 0
    start_time = time.time()

    # Start capturing video input from the camera
    cap = cv2.VideoCapture(camera_id)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

    # Visualization parameters
    row_size = 20  # pixels
    left_margin = 24  # pixels
    text_color = (0, 0, 255)  # red
    font_size = 1
    font_thickness = 1
    classification_results_to_show = 3
    fps_avg_frame_count = 10
    keypoint_detection_threshold_for_classifier = 0.1
    classifier = None
    
    #key = cv2.waitKey(0)
    #cv2.startWindowThread()

    # Continuously capture images from the camera and run inference
    while cap.isOpened():
        success, image = cap.read()
        if not success:
          sys.exit(
              'ERROR: Unable to read from webcam. Please verify your webcam settings.'
          )

        counter += 1
        image = cv2.flip(image, 1)
        persons = pose_detector.detect(image)

        # Draw keypoints and edges on input image
        image = utils.visualize(image, persons)

        # Calculate the FPS
        if counter % fps_avg_frame_count == 0:
          end_time = time.time()
          fps = fps_avg_frame_count / (end_time - start_time)
          start_time = time.time()

        # Show the FPS
        fps_text = 'FPS = ' + str(int(fps))
        text_location = (left_margin, row_size)
        cv2.putText(image, fps_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                    font_size, text_color, font_thickness)

        # Stop the program if the q key is pressed.
        if cv2.waitKey(1) == ord('q'):
          break
        cv2.imshow(model, image)
        '''
        key = cv2.waitKey(0)

        if key == ord('q'):            #qを押した時の処理
            cv2.waitKey(1)
            cv2.destroyAllWindows()  
            cap.release()
            cv2.waitKey(1)
            '''
            
    cv2.destroyAllWindows()  
    cap.release()

In [3]:
myRun(0, 1280, 720)

SystemExit: ERROR: Unable to read from webcam. Please verify your webcam settings.

/Users/nori/.pyenv/versions/3.9.16/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#qを押すとクラッシュしないように閉じる版

# このバージョンから再開する 2024Mar29th
MoveNetMultiPoseのなかにトラッカーへのdetect結果の入力がある。
ここをいじって、たとえばboundの条件を満たす大きさの結果だけトラッカーにいれる、を考えられる

In [1]:
import cv2
import numpy as np
import serial
import time

import argparse
import logging
import sys
import utils

from ml import Classifier
from ml import Movenet
from ml import MoveNetMultiPose
from ml import Posenet

model = 'movenet_multipose'
pose_detector = MoveNetMultiPose(model,'bounding_box',512)

width = 1024
height = 720
camera_id = "video2-720p-20fps.mp4"
#camera_id = "video0-10fps.mp4"
#camera_id = 1

# Start capturing video input from the camera
cap = cv2.VideoCapture(camera_id)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

image = np.zeros((720, 1280,3), np.uint8)
cv2.putText(image, "hello", (10, 30),
               cv2.FONT_HERSHEY_PLAIN, 1.5,
               (255, 255, 255), 1, cv2.LINE_AA)

cv2.startWindowThread()
cv2.imshow(model, image)

# Visualization parameters
row_size = 20  # pixels
left_margin = 24  # pixels
text_color = (0, 0, 255)  # red
font_size = 1
font_thickness = 1
classification_results_to_show = 3
fps_avg_frame_count = 10
keypoint_detection_threshold_for_classifier = 0.1
classifier = None

start_time = time.time()

while cap.isOpened():
    end_time = time.time()
    
    time_diff = end_time - start_time #sec
    fps = 1.0 / time_diff
        
    start_time = time.time()
    
    success, image = cap.read()
    if not success:
      sys.exit(
          'ERROR: Unable to read from webcam. Please verify your webcam settings.'
      )

    image = cv2.flip(image, 1)
    persons = pose_detector.detect(image)

    # Draw keypoints and edges on input image
    image = utils.visualize(image, persons)
    
    #canvas = np.zeros((720, 1280,3), np.uint8)

    # Show the FPS
    fps_text = 'FPS = ' + str(int(fps))
    text_location = (left_margin, row_size)
    cv2.putText(image, fps_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                font_size, text_color, font_thickness)
    time.sleep(0.01)
    
    cv2.imshow(model, image)
    
    key = cv2.waitKey(1)

    if key == ord('q'):            #qを押した時の処理
        cv2.waitKey(1)
        cv2.destroyAllWindows()  
        cap.release()
        cv2.waitKey(1)
        break

SystemExit: ERROR: Unable to read from webcam. Please verify your webcam settings.

/Users/nori/.pyenv/versions/3.9.16/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


課題：opencvで画面を出して、配列５m分を用意して表示する

In [1]:
import cv2
import numpy as np
import serial
import time
import logging
import sys
import utils

from typing import List, Tuple
from data import Person

from ml import Classifier
from ml import Movenet
from ml import MoveNetMultiPose
from ml import Posenet

class Body:
    id = 0
    bounding_box_start_point = (10,10)
    bounding_box_end_point = (20,20)
    

#1 do inference 
#2 convert the results to the list of Body objects
def process_bodies(
    list_persons: List[Person],
    keypoint_threshold: float = 0.8,
    instance_threshold: float = 0.8,
)-> List[Body]:
    
    bodies = [];
    for person in list_persons:
        if person.score < instance_threshold:
            continue
        keypoints = person.keypoints
        bounding_box = person.bounding_box
        
        # Draw bounding_box with multipose
        if bounding_box is not None:
            body = Body()
            body.bounding_box_start_point  =  bounding_box.start_point
            body.bounding_box_end_point = bounding_box.end_point
            body.id = person.id
            bodies.append(body)  
    return bodies

def draw_bodies(
    image: np.ndarray,
    list_bodies: List[Body],
) -> np.ndarray:
    for body in list_bodies:
        print("startp:",body.bounding_box_start_point[0],body.bounding_box_start_point[1])
        cv2.rectangle(image,body.bounding_box_start_point,body.bounding_box_end_point, (255, 255, 255), 1)
        id_text = 'id = ' + str(body.id)
        cv2.putText(image, id_text, body.bounding_box_start_point, cv2.FONT_HERSHEY_PLAIN, 1,(127, 127, 127), 1)
    return image

def draw_bodies_line(
    image: np.ndarray,
    list_bodies: List[Body],
) -> np.ndarray:
    temp_height = int(height/2)
    for body in list_bodies:
        cv2.rectangle(image,(body.bounding_box_start_point[0],temp_height),(body.bounding_box_end_point[0],temp_height), (255, 255, 255), 2)
        id_text = 'id = ' + str(body.id)
        cv2.putText(image, id_text, (body.bounding_box_start_point[0],temp_height), cv2.FONT_HERSHEY_PLAIN, 1,(127, 127, 127), 1)
    return image

def process_leds():
    interval = width/led_length
    leds_temp = [0] * led_length
    for body in bodies:
        start = body.bounding_box_start_point[0]
        end = body.bounding_box_end_point[0]
        start_led = int(start/interval)
        end_led = int(end/interval)
        
        for num in range(start_led,end_led):
            leds_temp[num] = 254
            
    ledss.append(leds_temp)
    ledss.pop(0)
    
    for num in range(led_length):
        colors = [0] * led_average;
        for index in range(led_average):
            colors[index] = ledss[index][num]
        color = np.mean(colors)
        leds[num] = color

def draw_leds(image: np.ndarray) -> np.ndarray:
    temp_height = int(height/4)
    interval = width/led_length
    for num in range(led_length):
        color = leds[num]
        cv2.circle(image, (int(interval*num), temp_height), 2, (color, color, color,127), thickness=-1)
    return image
        
    
model = 'movenet_multipose'
pose_detector = MoveNetMultiPose(model,'bounding_box',512)

width = 1024
height = 720
camera_id = 0

led_length = 150;  #5m 150
led_average = 5;
leds = [0] * led_length
ledss = [[0] * led_length] * led_average

# Start capturing video input from the camera
cap = cv2.VideoCapture(camera_id)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

image = np.zeros((height, width,3), np.uint8)
cv2.putText(image, "hello", (10, 30),
               cv2.FONT_HERSHEY_PLAIN, 1.5,
               (255, 255, 255), 1, cv2.LINE_AA)

cv2.startWindowThread()
cv2.imshow(model, image)

# Visualization parameters
row_size = 20  # pixels
left_margin = 24  # pixels
text_color = (0, 0, 255)  # red
font_size = 1
font_thickness = 1
classification_results_to_show = 3
fps_avg_frame_count = 10
keypoint_detection_threshold_for_classifier = 0.1
classifier = None

start_time = time.time()

while cap.isOpened():
    end_time = time.time()
    
    time_diff = end_time - start_time #sec
    fps = 1.0 / time_diff
        
    start_time = time.time()
    
    success, image = cap.read()
    if not success:
      sys.exit(
          'ERROR: Unable to read from webcam. Please verify your webcam settings.'
      )

    image = cv2.flip(image, 1)
    persons = pose_detector.detect(image)

    # Draw keypoints and edges on input image
    #image = utils.visualize(image, persons,keypoint_threshold=0.2,instance_threshold=0.2)
    bodies = process_bodies(persons,keypoint_threshold=0.2,instance_threshold=0.2)
    image = draw_bodies(image, bodies)
    image = draw_bodies_line(image, bodies)
    process_leds()
    image = draw_leds(image)

    # Show the FPS
    fps_text = 'FPS = ' + str(int(fps))
    text_location = (left_margin, row_size)
    cv2.putText(image, fps_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                font_size, text_color, font_thickness)
    
    cv2.imshow(model, image)
    time.sleep(0.02)
    key = cv2.waitKey(1)

    if key == ord('q'):            #qを押した時の処理
        cv2.waitKey(1)
        cv2.destroyAllWindows()  
        cap.release()
        cv2.waitKey(1)
        break
        

    

startp: 639 472
startp: 655 477
startp: 617 446
startp: 587 1
startp: 572 1
startp: 576 1
startp: 565 1
startp: 574 0
startp: 565 0
startp: 576 0
startp: 565 0
startp: 567 0
startp: 566 0
startp: 563 0
startp: 564 0
startp: 570 0
startp: 562 2
startp: 564 2
startp: 563 1
startp: 564 2
startp: 565 2
startp: 572 1
startp: 565 1
startp: 561 2
startp: 570 1
startp: 561 1
startp: 561 1
startp: 570 2
startp: 572 0
startp: 567 1
startp: 562 1
startp: 564 1
startp: 585 2
startp: 589 2
startp: 579 1
startp: 574 1
startp: 574 3
startp: 562 0
startp: 572 0
startp: 573 1
startp: 579 0
startp: 582 0
startp: 579 2
startp: 584 0
startp: 583 0
startp: 582 1
startp: 574 1
startp: 583 1
startp: 572 1
startp: 578 2
startp: 562 1
startp: 571 1
startp: 576 1
startp: 572 1
startp: 576 0
startp: 792 5
startp: 803 0
startp: 776 5
startp: 780 5
startp: 784 4
startp: 793 0
startp: 776 4
startp: 780 4
startp: 783 0
startp: 791 0
startp: 789 2
startp: 790 1
startp: 789 1
startp: 762 40
startp: 871 131
startp: 870

2024-02-09 13:10:32.741 python3.9[32331:1589829] _TIPropertyValueIsValid called with 4 on nil context!
2024-02-09 13:10:32.741 python3.9[32331:1589829] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2024-02-09 13:10:32.741 python3.9[32331:1589829] Text input context does not respond to _valueForTIProperty:
2024-02-09 13:10:32.744 python3.9[32331:1589829] _TIPropertyValueIsValid called with 4 on nil context!
2024-02-09 13:10:32.744 python3.9[32331:1589829] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2024-02-09 13:10:32.744 python3.9[32331:1589829] Text input context does not respond to _valueForTIProperty:


課題：Serialに繋いで、バイト配列を送る

In [3]:
import cv2
import numpy as np
import serial
import time
import logging
import sys
import utils

from typing import List, Tuple
from data import Person

from ml import Classifier
from ml import Movenet
from ml import MoveNetMultiPose
from ml import Posenet

class Led:
    def __init__(self):
        self.r=0
        self.g=0
        self.b=0
        self.w=0
    
    def clear(self):
        self.r=0
        self.g=0
        self.b=0
        self.w=0        

class Leds:
    def __init__(self,length):
        self.length = length
        self.leds = []
        for i in range(length):
            led = Led()
            self.leds.append(led)
    def clear(self):
        for led in self.leds:
            led.clear()
    def set_white(self,index,white):
        self.leds[index].w = white
    def set_red(self,index,red):
        self.leds[index].r = red
        
class Ledss:
    def __init__(self,length,depth):
        self.length = length
        self.leds = []
        self.ledss = []
        self.index = 0;
        self.depth = depth
        
        for i in range(length):
            led = Led()
            self.leds.append(led)
        for i in range(depth):
            leds = Leds(length)
            self.ledss.append(leds)
    def process(self):
        self.index+=1
        if self.index==self.depth:
            self.index = 0
            
        # culc averate
        depth = self.depth
        for i in range(self.length):
            r=0
            g=0
            b=0
            w=0
            
            for h in range(depth):
                leds_temp = self.ledss[h]
                led_temp = leds_temp.leds[i]
                r+= led_temp.r
                g+= led_temp.g
                b+= led_temp.b
                w+= led_temp.w
            self.leds[i].r = int(r/depth)
            self.leds[i].g = int(g/depth)
            self.leds[i].b = int(b/depth)
            self.leds[i].w = int(w/depth)
                
    def clear(self):
        self.ledss[self.index].clear()
    def set_white(self,index,white):
        self.ledss[self.index].set_white(index,white)
    def set_red(self,index,red):
        self.ledss[self.index].set_red(index,red)
        

class Part:
    def __init__(self,):
        self.isExist = False
        self.pos=(0,0)
    
class Bound:
    def __init__(self):
        self.x=0
        self.y=0
        self.w=0
        self.h=0

class Body(object):
    def __init__(self,person):
        self.person= person
        self.id = person.id
        self.keypoints = person.keypoints   
        self.bound = Bound()

        self.nose=Part()
        self.leye=Part()
        self.reye=Part()
        self.lear=Part()
        self.rear=Part()
        self.lshoulder=Part()
        self.rshoulder=Part()
        self.lelbow = Part()
        self.relbow = Part()
        self.lhand = Part()
        self.rhand = Part()
        self.lhip = Part()
        self.rhip = Part()
        self.lknee = Part()
        self.rknee = Part()
        self.lfoot = Part()
        self.rfoot = Part()
        self.parts = [
            self.nose,
            self.leye,
            self.reye,
            self.lear,
            self.rear,
            self.lshoulder,
            self.rshoulder,
            self.lelbow,
            self.relbow,
            self.lhand,
            self.rhand,
            self.lhip,
            self.rhip,
            self.lknee,
            self.rknee,
            self.lfoot,
            self.rfoot
        ]
    
    def process(self):
        #culc bound box
        minx = 10000
        miny = 10000
        maxx = 0
        maxy = 0
        for part in self.parts:
            if(part.isExist):
                x = part.pos[0]
                y = part.pos[1]
                if maxx<x:
                    maxx = x
                if maxy<y:
                    maxy = y
                if x<minx:
                    minx = x
                if y<miny:
                    miny = y
        self.bound.x = minx
        self.bound.y = miny
        self.bound.w = maxx-minx
        self.bound.h = maxy-miny
                
        
#1 do inference 
#2 convert the results to the list of Body objects
def process_bodies(
    list_persons: List[Person],
    keypoint_threshold: float = 0.1,
    instance_threshold: float = 0.1,
)-> List[Body]:
    
    bodies = []
    for person in list_persons:
        # print(type(person))
        
        if person.score < instance_threshold:
            continue
        #key points
        body = Body(person)
        for i in range(len(body.keypoints)):
            point = body.keypoints[i]
            if point.score >= keypoint_threshold:
                if(i==0):
                    body.nose.pos = point.coordinate
                    body.nose.isExist = True
                if(i==1):
                    body.leye.pos = point.coordinate
                    body.leye.isExist = True
                if(i==2):
                    body.reye.pos = point.coordinate
                    body.reye.isExist = True
                if(i==3):
                    body.lear.pos = point.coordinate
                    body.lear.isExist = True
                if(i==4):
                    body.rear.pos = point.coordinate
                    body.rear.isExist = True
                if(i==5):
                    body.lshoulder.pos = point.coordinate
                    body.lshoulder.isExist = True
                if(i==6):
                    body.rshoulder.pos = point.coordinate
                    body.rshoulder.isExist = True
                if(i==7):
                    body.lelbow.pos = point.coordinate
                    body.lelbow.isExist = True
                if(i==8):
                    body.relbow.pos = point.coordinate
                    body.relbow.isExist = True
                if(i==9):
                    body.lhand.pos = point.coordinate
                    body.lhand.isExist = True
                if(i==10):
                    body.rhand.pos = point.coordinate
                    body.rhand.isExist = True
                if(i==11):
                    body.lhip.pos = point.coordinate
                    body.lhip.isExist = True
                if(i==12):
                    body.rhip.pos = point.coordinate
                    body.rhip.isExist = True
                if(i==13):
                    body.lknee.pos = point.coordinate
                    body.lknee.isExist = True
                if(i==14):
                    body.rknee.pos = point.coordinate
                    body.rknee.isExist = True
                if(i==15):
                    body.lfoot.pos = point.coordinate
                    body.lfoot.isExist = True
                if(i==16):
                    body.rfoot.pos = point.coordinate
                    body.rfoot.isExist = True
        body.process()
        bodies.append(body)  
    return bodies

def draw_bodies(
    image: np.ndarray,
    list_bodies: List[Body],
) -> np.ndarray:
    for body in list_bodies:
        id_text = 'id = ' + str(body.id)
        cv2.putText(image, id_text, (body.bound.x,body.bound.y), cv2.FONT_HERSHEY_PLAIN, 1,(127, 127, 127), 1)
        if(body.nose.isExist):
            cv2.circle(image, body.nose.pos, 10, (0,0,255), thickness=-1)
        if(body.leye.isExist):
            cv2.circle(image, body.leye.pos, 10, (127,127,255), thickness=-1)
        if(body.reye.isExist):
            cv2.circle(image, body.reye.pos, 10, (127,127,255), thickness=-1)
        if(body.lear.isExist):
            cv2.circle(image, body.lear.pos, 10, (127,127,200), thickness=-1)
        if(body.rear.isExist):
            cv2.circle(image, body.rear.pos, 10, (127,127,200), thickness=-1)
        if(body.lshoulder.isExist):
            cv2.circle(image, body.lshoulder.pos, 10, (127,127,200), thickness=-1)
        if(body.rshoulder.isExist):
            cv2.circle(image, body.rshoulder.pos, 10, (127,127,200), thickness=-1)
            
        index = 0
        for part in body.parts:
            if index>6:
                if(part.isExist):
                    cv2.circle(image, part.pos, 10, (200,127,127), thickness=-1)
            index = index+1
            
            
        #print("startp:",body.bounding_box_start_point[0],body.bounding_box_start_point[1])
        cv2.rectangle(image,(body.bound.x,body.bound.y),(body.bound.x+body.bound.w,body.bound.y+body.bound.h), (255, 255, 255), 1)
    return image

def draw_bodies_line(
    image: np.ndarray,
    list_bodies: List[Body],
) -> np.ndarray:
    temp_height = int(height/2)
    for body in list_bodies:
        cv2.rectangle(image,(body.bound.x,temp_height),(body.bound.x + body.bound.w,temp_height), (255, 255, 255), 2)
        cv2.rectangle(image,(body.leye.pos[0],temp_height),(body.leye.pos[0]+10,temp_height), (255, 0,0), 2)
        cv2.rectangle(image,(body.reye.pos[0],temp_height),(body.reye.pos[0]+10,temp_height), (255, 0,0), 2)
        
        id_text = 'id = ' + str(body.id)
        cv2.putText(image, id_text,(body.bound.x,temp_height), cv2.FONT_HERSHEY_PLAIN, 1,(127, 127, 127), 1)
    return image

def process_leds():
    interval = width/led_length
    #leds_temp = [0] * led_length
    ledss.clear()
    for body in bodies:
        start = body.bound.x
        end = body.bound.x+body.bound.w
        start_led = int(start/interval)
        end_led = int(end/interval)
        l_eye = int(body.leye.pos[0]/interval)
        r_eye = int(body.reye.pos[0]/interval)
        
        for num in range(start_led,end_led):
            ledss.set_white(num,254)
        ledss.set_red(l_eye,254)
        ledss.set_red(r_eye,254)
    ledss.process()

def draw_leds(image: np.ndarray) -> np.ndarray:
    temp_height = int(height/4)
    interval = width/led_length
    for index in range(led_length):
        led = ledss.leds[index]
        cv2.circle(image, (int(interval*index), temp_height), 2, (led.w, led.w, led.w), thickness=-1)
        cv2.circle(image, (int(interval*index), temp_height-6), 2, (led.b, led.g, led.r), thickness=-1)
    return image


#utils.visualize
# Draw keypoints and edges on input image
#image = utils.visualize(image, persons)
        
    
model = 'movenet_multipose'
pose_detector = MoveNetMultiPose(model,'bounding_box',512)

width = 1024
height = 720
#width = 1920
#height = 1080
camera_id = 0

led_length = 150;  #5m 150
led_average = 5;

leds = Leds(led_length)
ledss = Ledss(led_length,led_average)

# Start capturing video input from the camera
cap = cv2.VideoCapture(camera_id)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

image = np.zeros((height, width,3), np.uint8)
cv2.putText(image, "hello", (10, 30),
               cv2.FONT_HERSHEY_PLAIN, 1.5,
               (255, 255, 255), 1, cv2.LINE_AA)

cv2.startWindowThread()
cv2.imshow(model, image)

# Visualization parameters
#row_size = 20  # pixels
#left_margin = 24  # pixels
#text_color = (0, 0, 255)  # red
#font_size = 1
#font_thickness = 1
classification_results_to_show = 10
#fps_avg_frame_count = 10
#keypoint_detection_threshold_for_classifier = 0.1
#classifier = None

start_time = time.time()

while cap.isOpened():
    end_time = time.time()
    
    time_diff = end_time - start_time #sec
    fps = 1.0 / time_diff
        
    start_time = time.time()
    
    success, image = cap.read()
    if not success:
      sys.exit(
          'ERROR: Unable to read from webcam. Please verify your webcam settings.'
      )

    image = cv2.flip(image, 1)
    persons = pose_detector.detect(image)
    
    print("persons:"+str(len(persons)))

    # Draw keypoints and edges on input image
    #image = utils.visualize(image, persons,keypoint_threshold=0.2,instance_threshold=0.2)
    bodies = process_bodies(persons,keypoint_threshold=0.1,instance_threshold=0.1)
    image = draw_bodies(image, bodies)
    #image = draw_bodies_line(image, bodies)
    #process_leds()
    #image = draw_leds(image)

    # Show the FPS
    fps_text = 'FPS = ' + str(int(fps))
    text_location = (10,20)
    cv2.putText(image, fps_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                1, (255,255,255), 1)
    
    cv2.imshow(model, image)
    time.sleep(0.02)
    key = cv2.waitKey(1)

    if key == ord('q'):            #qを押した時の処理
        cv2.waitKey(1)
        cv2.destroyAllWindows()  
        cap.release()
        cv2.waitKey(1)
        break

persons:0
persons:0
persons:0
persons:0
persons:0
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:0
persons:1
persons:0
persons:1
persons:1
persons:1
persons:2
persons:1
persons:1
persons:1
persons:1
persons:2
persons:2
persons:1
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:1
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:2
persons:1
persons:2
persons:2
